In [74]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
import datetime
import pandas as pd
import numpy as np
from functools import partial
from dateutil import parser as dtparser

from utils.datafetch import *
from utils.vectorized_funs import *
from utils.datapipe import *
from FintechCapstone import FinCapstone

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


In [79]:
date_from = '2009-01-01'
date_to = '2017-03-01'
timespan = {
    "short_term": [1, 5]
    ,"medium_term": [40]
    ,"long_term": [90]
}

timespan_ab = {
    "short_term": []
    ,"medium_term": [30]
    ,"long_term": []
}

trial = FinCapstone(ticker_list_samplesize=10, timespan=timespan, timespan_ab=timespan_ab, date_from=date_from)

amex has 400 tickers.
nasdaq has 3081 tickers.
nyse has 3299 tickers.


Final dataset has 6780 records
(NOK) Final dataset has duplicate tickers (6779 distinct)


In [5]:
trial.run_initial_dataload()

(1/10) Recv. and Stored XXII
(2/10) Recv. and Stored FAX
(3/10) Recv. and Stored IAF
(4/10) Recv. and Stored CH
(5/10) ERROR receiving ABE           
(6/10) Recv. and Stored FCO
(7/10) Recv. and Stored IF
(8/10) Recv. and Stored ISL
(9/10) Recv. and Stored ACU
(10/10) Recv. and Stored AIII
Took 0:00:05.270377


In [81]:
trial.feature_engineering(feature_set="baseline")



 - XXII - 

| | START     - 2017-04-03 20:11:52.624943
\ / RETURNS - 0:00:00.048305
 V  END       - 2017-04-03 20:11:52.673328 (TOOK 0:00:00.048385)


 - FAX - 

| | START     - 2017-04-03 20:11:52.754377
\ / RETURNS - 0:00:00.045577
 V  END       - 2017-04-03 20:11:52.800023 (TOOK 0:00:00.045646)


 - IAF - 

| | START     - 2017-04-03 20:11:52.910805
\ / RETURNS - 0:00:00.045357
 V  END       - 2017-04-03 20:11:52.956220 (TOOK 0:00:00.045415)


 - CH - 

| | START     - 2017-04-03 20:11:53.056091
\ / RETURNS - 0:00:00.046584
 V  END       - 2017-04-03 20:11:53.102769 (TOOK 0:00:00.046678)


 - FCO - 

| | START     - 2017-04-03 20:11:53.202890
\ / RETURNS - 0:00:00.045100
 V  END       - 2017-04-03 20:11:53.248045 (TOOK 0:00:00.045155)


 - IF - 

| | START     - 2017-04-03 20:11:53.341788
\ / RETURNS - 0:00:00.045613
 V  END       - 2017-04-03 20:11:53.387463 (TOOK 0:00:00.045675)


 - ISL - 

| | START     - 2017-04-03 20:11:53.486923
\ / RETURNS - 0:00:00.046559
 V  END       - 

In [80]:
trial.load_train_eval_baseline()

array([0.5891719764205301, 0.5318471356561989, 0.49363057286876022,
       0.49681528738349867, 0.49681528890208831, 0.50636942713123978,
       0.49044585949296404, 0.48726114687646271, 0.59554140127388533], dtype=object)

array(['XXII', 'FAX', 'IAF', 'CH', 'FCO', 'IF', 'ISL', 'ACU', 'AIII'], dtype=object)